In [1]:
import pandas as pd
import os
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score, precision_recall_curve, average_precision_score, mean_squared_error, r2_score, mean_absolute_error
from scipy.stats import pearsonr

import numpy as np

# preds

# Load the training set of meta-model
lipo_chemberta2_valid2 = pd.read_csv('./chemberta2/results/lipo/chemberta2_valid2_lipo_1_predictions.csv')
lipo_molformer_valid2 = pd.read_csv('./molformer/results/lipo/molformer_lipo_valid2_1.csv')
lipo_molbert_valid2 = pd.read_csv('./molbert/results/lipo/molbert_valid2_lipo_1.csv')

# Load the test data for each model
lipo_chemberta2_test = pd.read_csv('./chemberta2/results/lipo/chemberta2_test_lipo_1_predictions.csv')
lipo_molformer_test = pd.read_csv('./molformer/results/lipo/molformer_lipo_test_1.csv')
lipo_molbert_test = pd.read_csv('./molbert/results/lipo/molbert_test_lipo_1.csv')

train_mean = 2.1741626984126983
train_sd = 1.2060271152247222

# features

lipo_chemberta2_features_valid2 = pd.read_csv('./chemberta2/features/lipo/chemberta2_valid2_lipo_1_features.csv')
lipo_chemberta2_features_test = pd.read_csv('./chemberta2/features/lipo/chemberta2_test_lipo_1_features.csv')

lipo_molformer_features_valid2 = pd.read_csv('./molformer/features/lipo/molformer_valid2_lipo_1_features.csv')
lipo_molformer_features_test = pd.read_csv('./molformer/features/lipo/molformer_test_lipo_1_features.csv')

lipo_molbert_features_valid2 = pd.read_csv('./molbert/features/lipo/molbert_valid2_lipo_1_features.csv')
lipo_molbert_features_test = pd.read_csv('./molbert/features/lipo/molbert_test_lipo_1_features.csv')

For Lipo (Regression)

In [2]:
# Preparing the actual and predicted values
# Chemberta2
lipo_chemberta_actual = lipo_chemberta2_test['target'] 
lipo_chemberta_pred = lipo_chemberta2_test['pred_raw']

# Molformer
lipo_molformer_actual = lipo_molformer_test['target']
lipo_molformer_pred = lipo_molformer_test['pred_raw']

# molbert
lipo_molbert_actual = lipo_molbert_test['target']
lipo_molbert_pred = lipo_molbert_test['pred_raw']

In [3]:
# Calculating metrics
lipo_metrics_results = {}

for model_name, actual, pred in [("Chemberta2", lipo_chemberta_actual, lipo_chemberta_pred),
                                 ("Molformer", lipo_molformer_actual, lipo_molformer_pred),
                                 ("Molbert", lipo_molbert_actual, lipo_molbert_pred)]:
    lipo_metrics_results[model_name] = {
        "MAE": mean_absolute_error(actual, pred),
        "RMSE": np.sqrt(mean_squared_error(actual, pred)),
        "R2 Score": r2_score(actual, pred),
        "Correlation": pearsonr(actual, pred)[0]  # Only record the correlation coefficient
    }

lipo_metrics_results

{'Chemberta2': {'MAE': 0.4868551307755036,
  'RMSE': 0.6482952518588416,
  'R2 Score': 0.707140489497853,
  'Correlation': 0.8472657204543116},
 'Molformer': {'MAE': 0.49439111716666667,
  'RMSE': 0.6626307207706376,
  'R2 Score': 0.6940455432003805,
  'Correlation': 0.8377379022328877},
 'Molbert': {'MAE': 0.5154242747119047,
  'RMSE': 0.6940250633438312,
  'R2 Score': 0.6643675374778397,
  'Correlation': 0.8171700472906422}}

In [4]:
# standardized valid2 labels
lipo_y_ensemble_valid2 = (lipo_chemberta2_valid2['target'] - train_mean)/train_sd

# Create the features for the ensemble from the prediction probabilities of being in class 1
lipo_X_ensemble_valid2 = pd.concat([
    lipo_chemberta2_valid2['pred_z'] - lipo_y_ensemble_valid2,
    lipo_molformer_valid2['pred_z'] - lipo_y_ensemble_valid2, 
    lipo_molbert_valid2['pred_z'] - lipo_y_ensemble_valid2,
    # add features from training set
    lipo_chemberta2_valid2['pred_z'],
    lipo_molformer_valid2['pred_z'],
    lipo_molbert_valid2['pred_z']
], axis=1)

# change feature names of the ensemble so that they are unique
lipo_X_ensemble_valid2.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

In [5]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Compute residuals
chemberta_y_residual = lipo_chemberta2_valid2['pred_z'] - lipo_y_ensemble_valid2
molformer_y_residual = lipo_molformer_valid2['pred_z'] - lipo_y_ensemble_valid2
molbert_y_residual = lipo_molbert_valid2['pred_z'] - lipo_y_ensemble_valid2

# For input features, we use pred_z and the other features
chemberta_X = pd.concat([lipo_chemberta2_valid2['pred_z'], 
                         lipo_chemberta2_features_valid2.iloc[:, 2:]], axis=1)
molformer_X = pd.concat([lipo_molformer_valid2['pred_z'], 
                         lipo_molformer_features_valid2.iloc[:, 1:]], axis=1)
molbert_X = pd.concat([lipo_molbert_valid2['pred_z'], 
                       lipo_molbert_features_valid2.iloc[:, 1:]], axis=1)

# Standardize each dataset
scaler_chemberta = StandardScaler().fit(chemberta_X)
scaler_molformer = StandardScaler().fit(molformer_X)
scaler_molbert = StandardScaler().fit(molbert_X)

chemberta_X_scaled = scaler_chemberta.transform(chemberta_X)
molformer_X_scaled = scaler_molformer.transform(molformer_X)
molbert_X_scaled = scaler_molbert.transform(molbert_X)

# Ensure that X_scaled and y_residual are numpy arrays
chemberta_X_scaled = np.array(chemberta_X_scaled)
molformer_X_scaled = np.array(molformer_X_scaled)
molbert_X_scaled = np.array(molbert_X_scaled)

chemberta_y_residual = np.array(chemberta_y_residual)
molformer_y_residual = np.array(molformer_y_residual)
molbert_y_residual = np.array(molbert_y_residual)

torch.manual_seed(0)

# Define custom RMSE loss function
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Define the neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]  # Output is continuous, no activation function
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Objective function for Bayesian optimization
def objective(params, X, y):
    kf = KFold(n_splits=5)
    rmses = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X[train_index], X[val_index]
        y_train, y_val = y[train_index], y[val_index]

        train_dataset = TensorDataset(torch.tensor(X_train.astype(np.float32)), 
                                      torch.tensor(y_train.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=int(params['num_layers']), 
                         num_neurons=int(params['num_neurons']), dropout_rate=params['dropout_rate'])
        criterion = RMSELoss()  # Use RMSELoss for regression
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):  # Set number of epochs
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            X_val_tensor = torch.tensor(X_val.astype(np.float32))
            y_val_tensor = torch.tensor(y_val.astype(np.float32)).unsqueeze(-1)
            outputs = model(X_val_tensor)
            rmse = np.sqrt(mean_squared_error(y_val_tensor.numpy(), outputs.numpy()))
            rmses.append(rmse)

    avg_rmse = np.mean(rmses)
    return {'loss': avg_rmse, 'status': STATUS_OK}  # Minimize RMSE

# Hyperparameter space for optimization
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

# Scaled input data (after standardization) and corresponding target (binary cross-entropy loss)
datasets = {
    'chemberta': (chemberta_X_scaled, chemberta_y_residual),
    'molformer': (molformer_X_scaled, molformer_y_residual),
    'molbert': (molbert_X_scaled, molbert_y_residual)
}

# Run Bayesian optimization for each model
best_params = {}
trials_dict = {}

for dataset_name, (X_scaled, y_residual) in datasets.items():
    print(f"Optimizing for {dataset_name}...")
    trials = Trials()
    
    best_params[dataset_name] = fmin(fn=lambda params: objective(params, X_scaled, y_residual),
                                     space=space,
                                     algo=tpe.suggest,
                                     max_evals=50,
                                     trials=trials,
                                     rstate=np.random.default_rng(0),  # Seed for reproducibility in hyperopt
                                     early_stop_fn=no_progress_loss(10))
    
    trials_dict[dataset_name] = trials
    print(f"Best hyperparameters for {dataset_name}: {best_params[dataset_name]}")

Optimizing for chemberta...
 20%|██        | 10/50 [01:38<06:34,  9.86s/trial, best loss: 0.5526280999183655]
Best hyperparameters for chemberta: {'dropout_rate': 0.4175299871186762, 'learning_rate': 0.00024043206714908388, 'num_layers': 2.0, 'num_neurons': 180.0}
Optimizing for molformer...
 20%|██        | 10/50 [01:47<07:11, 10.78s/trial, best loss: 0.5579864382743835]
Best hyperparameters for molformer: {'dropout_rate': 0.4175299871186762, 'learning_rate': 0.00024043206714908388, 'num_layers': 2.0, 'num_neurons': 180.0}
Optimizing for molbert...
 30%|███       | 15/50 [02:52<06:43, 11.53s/trial, best loss: 0.5509543418884277]
Best hyperparameters for molbert: {'dropout_rate': 0.36905642941420785, 'learning_rate': 0.0005609649510850792, 'num_layers': 5.0, 'num_neurons': 147.0}


In [6]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np
from sklearn.metrics import mean_squared_error

# Custom RMSE loss function
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Neural network model
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(num_layers - 1):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]  # Output layer
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Function to train the model using the best hyperparameters
def train_model(X_train, y_train, best_params):
    # Create DataLoader for training data
    train_dataset = TensorDataset(torch.tensor(X_train.astype(np.float32)), 
                                  torch.tensor(y_train.astype(np.float32)).unsqueeze(1))
    train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

    # Initialize the model with the best hyperparameters
    model = SimpleNN(input_size=X_train.shape[1],
                     num_layers=int(best_params['num_layers']),
                     num_neurons=int(best_params['num_neurons']),
                     dropout_rate=best_params['dropout_rate'])
    
    # Define the loss and optimizer
    criterion = RMSELoss()
    optimizer = optim.Adam(model.parameters(), lr=best_params['learning_rate'])

    # Train the model
    model.train()
    for epoch in range(100):  # Training for 100 epochs
        for inputs, targets in train_loader:
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()

    return model

# Load your best hyperparameters from the optimization step
# Replace these with the actual best hyperparameters you found
best_params_chemberta = best_params['chemberta']
best_params_molformer = best_params['molformer']
best_params_molbert = best_params['molbert']

# Train the models for each dataset
best_model_chemberta = train_model(chemberta_X_scaled, chemberta_y_residual, best_params_chemberta)
best_model_molformer = train_model(molformer_X_scaled, molformer_y_residual, best_params_molformer)
best_model_molbert = train_model(molbert_X_scaled, molbert_y_residual, best_params_molbert)

In [7]:
# ChemBERTa
chemberta_X_test = pd.concat([lipo_chemberta2_test['pred_z'], 
                              lipo_chemberta2_features_test.iloc[:, 2:]], axis=1)

# MolFormer
molformer_X_test = pd.concat([lipo_molformer_test['pred_z'], 
                              lipo_molformer_features_test.iloc[:, 1:]], axis=1)

# MolBERT
molbert_X_test = pd.concat([lipo_molbert_test['pred_z'], 
                            lipo_molbert_features_test.iloc[:, 1:]], axis=1)

# Scale the test data using the corresponding scalers
chemberta_X_test_scaled = scaler_chemberta.transform(chemberta_X_test)
molformer_X_test_scaled = scaler_molformer.transform(molformer_X_test)
molbert_X_test_scaled = scaler_molbert.transform(molbert_X_test)

# Ensure that X_scaled and y_residual are numpy arrays
chemberta_X_test_scaled = np.array(chemberta_X_test_scaled)
molformer_X_test_scaled = np.array(molformer_X_test_scaled)
molbert_X_test_scaled = np.array(molbert_X_test_scaled)

# Predict on the test sets (assuming your test sets are already preprocessed and scaled)
with torch.no_grad():
    chemberta_X_test_tensor = torch.tensor(chemberta_X_test_scaled.astype(np.float32))
    molformer_X_test_tensor = torch.tensor(molformer_X_test_scaled.astype(np.float32))
    molbert_X_test_tensor = torch.tensor(molbert_X_test_scaled.astype(np.float32))
    
    chemberta_pred_test = best_model_chemberta(chemberta_X_test_tensor).numpy()
    molformer_pred_test = best_model_molformer(molformer_X_test_tensor).numpy()
    molbert_pred_test = best_model_molbert(molbert_X_test_tensor).numpy()
        
    # Ensure the predictions are 1-dimensional by flattening if necessary
    chemberta_pred_test = chemberta_pred_test.flatten()
    molformer_pred_test = molformer_pred_test.flatten()
    molbert_pred_test = molbert_pred_test.flatten()
    
# The predictions are now stored in chemberta_pred_test, molformer_pred_test, molbert_pred_test

# Convert the predictions (numpy arrays) to pandas Series
chemberta_pred_test_series = pd.Series(chemberta_pred_test, name='residual_chemberta')
molformer_pred_test_series = pd.Series(molformer_pred_test, name='residual_molformer')
molbert_pred_test_series = pd.Series(molbert_pred_test, name='residual_molbert')

# Now concatenate the series with the test set probabilities
lipo_X_ensemble_test = pd.concat([
    chemberta_pred_test_series,                     # residual for Chemberta
    molformer_pred_test_series,                     # residual for Molformer
    molbert_pred_test_series,                       # residual for Molbert
    lipo_chemberta2_test['pred_z'],
    lipo_molformer_test['pred_z'],  
    lipo_molbert_test['pred_z']
], axis=1)

# Rename feature columns so that they are unique
lipo_X_ensemble_test.columns = ['residuals_chemberta', 'residuals_molformer', 'residuals_molbert', 'chemberta', 'molformer', 'molbert']

# True test labels
lipo_y_ensemble_test = lipo_chemberta2_test['target']

In [8]:
# scale the features
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
lipo_X_ensemble_valid2_scaled = scaler.fit_transform(lipo_X_ensemble_valid2)
lipo_X_ensemble_test_scaled = scaler.transform(lipo_X_ensemble_test)

lipo_X_ensemble_valid2_scaled = pd.DataFrame(lipo_X_ensemble_valid2_scaled, columns=lipo_X_ensemble_valid2.columns)
lipo_X_ensemble_test_scaled = pd.DataFrame(lipo_X_ensemble_test_scaled, columns=lipo_X_ensemble_test.columns)

In [9]:
lipo_X_ensemble_valid2_selected = lipo_X_ensemble_valid2_scaled
lipo_X_ensemble_test_selected = lipo_X_ensemble_test_scaled

# check shapes
print(lipo_X_ensemble_valid2_selected.shape)
print(lipo_X_ensemble_test_selected.shape)

(840, 6)
(420, 6)


In [10]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import KFold
import numpy as np
from sklearn.metrics import mean_squared_error
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from hyperopt.early_stop import no_progress_loss

# Set seeds for reproducibility
np.random.seed(0)
torch.manual_seed(0)

# Define RMSE loss
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss, self).__init__()
        self.mse = nn.MSELoss()

    def forward(self, y_pred, y_true):
        return torch.sqrt(self.mse(y_pred, y_true))

# Define the neural network model for regression
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(1, num_layers):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Hyperparameter space with hp.quniform for integer distribution
space = {
    'num_layers': hp.quniform('num_layers', 1, 5, 1),
    'num_neurons': hp.quniform('num_neurons', 16, 256, 1),
    'learning_rate': hp.loguniform('learning_rate', np.log(0.0001), np.log(0.01)),
    'dropout_rate': hp.uniform('dropout_rate', 0.0, 0.5)
}

# Global dataset variables assumed to be defined externally
X = lipo_X_ensemble_valid2_selected
y = lipo_y_ensemble_valid2

# Objective function for Bayesian optimization
def objective(params):
    params['num_layers'] = int(params['num_layers'])  # Ensure num_layers is an integer
    params['num_neurons'] = int(params['num_neurons'])  # Ensure num_neurons is an integer
    kf = KFold(n_splits=5)
    rmse_scores = []

    for train_index, val_index in kf.split(X):
        X_train, X_val = X.iloc[train_index], X.iloc[val_index]
        y_train, y_val = y.iloc[train_index], y.iloc[val_index]

        # Convert DataFrame to numpy arrays before making them PyTorch tensors
        train_dataset = TensorDataset(torch.tensor(X_train.values.astype(np.float32)), 
                                      torch.tensor(y_train.values.astype(np.float32)).unsqueeze(1))
        train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

        model = SimpleNN(input_size=X_train.shape[1], num_layers=params['num_layers'],
                         num_neurons=params['num_neurons'], dropout_rate=params['dropout_rate'])
        criterion = RMSELoss()
        optimizer = optim.Adam(model.parameters(), lr=params['learning_rate'])

        model.train()
        for epoch in range(100):
            for inputs, targets in train_loader:
                optimizer.zero_grad()
                outputs = model(inputs)
                loss = criterion(outputs, targets)
                loss.backward()
                optimizer.step()

        model.eval()
        with torch.no_grad():
            val_preds = model(torch.tensor(X_val.values.astype(np.float32))).squeeze(1)
            val_targets = torch.tensor(y_val.values.astype(np.float32))
            rmse = np.sqrt(mean_squared_error(val_targets.numpy(), val_preds.numpy()))
            rmse_scores.append(rmse)

    avg_rmse = np.mean(rmse_scores)
    return {'loss': avg_rmse, 'status': STATUS_OK} # Minimize RMSE

# Run Bayesian optimization
trials = Trials()
lipo_nn_best_params = fmin(fn=objective,
                           space=space,
                           algo=tpe.suggest,
                           max_evals=50,
                           trials=trials,
                           rstate=np.random.default_rng(0),  # Seed for hyperopt
                           early_stop_fn=no_progress_loss(10))

print("Best hyperparameters:", lipo_nn_best_params)


 74%|███████▍  | 37/50 [06:02<02:07,  9.80s/trial, best loss: 0.020164698362350464]
Best hyperparameters: {'dropout_rate': 0.0010624021276385111, 'learning_rate': 0.00042066760941452476, 'num_layers': 4.0, 'num_neurons': 235.0}


In [11]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
import numpy as np

torch.manual_seed(0)

# Define the neural network model again
class SimpleNN(nn.Module):
    def __init__(self, input_size, num_layers, num_neurons, dropout_rate):
        super(SimpleNN, self).__init__()
        layers = [nn.Linear(input_size, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        for _ in range(1, num_layers):
            layers += [nn.Linear(num_neurons, num_neurons), nn.ReLU(), nn.Dropout(dropout_rate)]
        
        layers += [nn.Linear(num_neurons, 1)]
        
        self.layers = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.layers(x)

# Define a function to compute RMSE
def compute_rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

# Convert parameters to the correct format if necessary
lipo_nn_best_params = {
    'num_layers':  int(lipo_nn_best_params['num_layers']),  # Extracted from Bayesian optimization results
    'num_neurons':  int(lipo_nn_best_params['num_neurons']),  # Extracted from Bayesian optimization results
    'dropout_rate': lipo_nn_best_params['dropout_rate'],  # Extracted from Bayesian optimization results
    'learning_rate': lipo_nn_best_params['learning_rate']  # Extracted from Bayesian optimization results
}

# Prepare datasets
X_train_tensor = torch.tensor(lipo_X_ensemble_valid2_selected.values.astype(np.float32))
y_train_tensor = torch.tensor(lipo_y_ensemble_valid2.values.astype(np.float32)).unsqueeze(1)
train_dataset = TensorDataset(X_train_tensor, y_train_tensor)
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)

X_test_tensor = torch.tensor(lipo_X_ensemble_test_selected.values.astype(np.float32))
y_test_tensor = torch.tensor(lipo_y_ensemble_test.values.astype(np.float32)).unsqueeze(1)

# Initialize the model
model = SimpleNN(input_size=lipo_X_ensemble_valid2_selected.shape[1], num_layers=lipo_nn_best_params['num_layers'],
                         num_neurons=lipo_nn_best_params['num_neurons'], dropout_rate=lipo_nn_best_params['dropout_rate'])
criterion = RMSELoss()
optimizer = optim.Adam(model.parameters(), lr=lipo_nn_best_params['learning_rate'])

# Training loop
model.train()
for epoch in range(100):  # Number of epochs can be adjusted
    for inputs, targets in train_loader:
        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

# Evaluation on test set
model.eval()
with torch.no_grad():
    outputs = model(X_test_tensor)
    predictions = outputs.squeeze(1).numpy() * train_sd + train_mean

    # Calculate metrics
    mae = mean_absolute_error(y_test_tensor.numpy(), predictions)
    rmse = compute_rmse(y_test_tensor.numpy(), predictions)
    r2 = r2_score(y_test_tensor.numpy(), predictions)
    correlation, _ = pearsonr(y_test_tensor.numpy().squeeze(1), predictions)

    lipo_nn_metrics = {
        'MAE': mae,
        'RMSE': rmse,
        'R2 Score': r2,
        'Correlation': correlation
    }

lipo_nn_metrics

{'MAE': 0.45634472,
 'RMSE': 0.6185407,
 'R2 Score': 0.7334060668945312,
 'Correlation': 0.8579666685749313}

In [12]:
# create a table to record all metrics for lipo
lipo_metrics_results["Neural Network"] = lipo_nn_metrics

lipo_metrics_df = pd.DataFrame(lipo_metrics_results).T
# keep 3 digits after the decimal point
lipo_metrics_df = lipo_metrics_df.round(3)

# export table to csv
lipo_metrics_df.to_csv('./split1_lipo_metrics_nn.csv')